# Model Analysis

#### Model Performance Accuracy
    Linear Regression: 22.89%
    Logistic Regression: 89.93%
    SVM: 93.41%
    NN: 96.25%
    
    KNN: Too many features for my device

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
df_train = pd.read_csv(path+'train_data.csv')
df_val = pd.read_csv(path+'val_data.csv')
df_test = pd.read_csv(path+'test_data.csv')

In [2]:
# Converting dataframes -> numpy arrays to train models off of 
train = df_train.values
val = df_val.values
test = df_test.values

Xtr = train[:,1:]
Ytr = train[:,0]
Xval = val[:,1:]
Yval = val[:,0]
Xte = test[:,1:]
Yte = test[:,0]

In [3]:
# Normalizing data to use simple Keras models
from sklearn import preprocessing
Xtr = preprocessing.normalize(Xtr)
Xval = preprocessing.normalize(Xval)
Xte = preprocessing.normalize(Xte)

In [14]:
from sklearn import linear_model
from sklearn.metrics import accuracy_score

regr = linear_model.LinearRegression()
regr.fit(Xtr, Ytr)
Yhat = np.floor(regr.predict(Xval))
print("Linear model accuracy: {0:.2f}%".format(accuracy_score(Yval, Yhat)*100))

Linear model accuracy: 24.53%


In [9]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(Xtr, Ytr)
Yhat = lr.predict(Xval)
print("Linear model accuracy: {0:.2f}%".format(accuracy_score(Yval, Yhat)*100))

Linear model accuracy: 89.93%


In [5]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier

Yhat = OneVsOneClassifier(LinearSVC(random_state=0)).fit(Xtr, Ytr).predict(Xval)
print("SVM model accuracy: {0:.2f}%".format(accuracy_score(Yval, Yhat)*100))

SVM model accuracy: 93.41%


In [6]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier()
clf.fit(Xtr, Ytr)
Yhat = clf.predict(Xval)
print("NN model accuracy: {0:.2f}%".format(accuracy_score(Yval, Yhat)*100))

NN model accuracy: 96.25%


KNN takes too long on my device due to all the features present in this problem

In [58]:
path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
test_dataframe = pd.read_csv(path+'test.csv')

In [61]:
path = '/Users/antonmax2/Documents/dev/data/digit_recognizer/'
train_dataframe = pd.read_csv(path+'train.csv')

In [ ]:
train_dataframe

In [59]:
# Removing all features from test.csv not evaluated in our model
train_NL = df_train
train_NL = train_NL.drop(['label'], axis=1)
for i in range(784):
    col = "pixel" + str(i)
    if col not in train_NL.columns:
        test_dataframe = test_dataframe.drop([col], axis=1)

In [ ]:
Yhat = OneVsOneClassifier(LinearSVC(random_state=0)).fit(X).predict(Xval)

*Train model on raw train set, and on non uniform set, and on pixels included